In [129]:
import pandas as pd
import numpy as np
import surprise as sp

rec_path = r'/Users/erik/Downloads/archive(1)/recommendations.csv'
games_path = r'/Users/erik/Downloads/archive(1)/games.csv'
users_path = r'/Users/erik/Downloads/archive(1)/users.csv'
games_metadata_path = r'/Users/erik/Downloads/archive(1)/games_metadata.json'

recommendations = pd.read_csv(rec_path)
games = pd.read_csv(games_path)
users = pd.read_csv(users_path)
games_metadata = pd.read_json(games_metadata_path, lines=True)

In [130]:
import random
import numpy as np

my_seed = 1
random.seed(my_seed)
np.random.seed(my_seed)

In [131]:
user_app_ratings = pd.merge(users.loc[users['reviews'] >= 20], recommendations, how="inner", on=["user_id"])
user_app_ratings['is_recommended'] = user_app_ratings['is_recommended'].map({False: 0, True: 1})
user_app_ratings

,user_id,products,reviews,app_id,helpful,funny,date,is_recommended,hours,review_id
0,1965432,702,32,264710,0,2,2018-05-30,1,33.9,2063767
1,1965432,702,32,239030,0,0,2013-12-12,1,10.3,4517660
2,1965432,702,32,250900,0,0,2017-02-17,1,61.6,5762464
3,1965432,702,32,607080,2,0,2021-12-25,1,19.0,6152399
4,1965432,702,32,335300,2,0,2017-01-01,1,164.0,6410467
...,...,...,...,...,...,...,...,...,...,...
429428,922219,72,25,1361000,28,19,2022-06-26,1,1.1,9366376
429429,922219,72,25,289650,0,0,2022-06-24,1,16.1,9639562
429430,922219,72,25,424840,0,0,2021-12-28,0,2.7,9697813
429431,922219,72,25,633230,0,0,2021-12-28,1,10.0,11229597


In [124]:
games

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,10090,Call of Duty: World at War,2008-11-18,True,False,False,Very Positive,92,37039,19.99,19.99,0.0,True
1,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
2,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
3,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
4,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48313,1803600,Gone Rogue,2023-03-01,True,False,False,Very Positive,96,65,12.49,12.49,0.0,True
48314,1811990,Wildfrost,2023-04-12,True,False,False,Mixed,64,1819,19.99,19.99,0.0,True
48315,1868120,BLACK ACE,2022-02-03,True,False,False,Mixed,44,76,0.00,0.00,0.0,True
48316,2206390,BandRoll,2023-04-14,True,False,False,Positive,96,26,1.59,1.99,20.0,True


In [25]:
user_app_ratings.loc[user_app_ratings['is_recommended'] == 0]

,user_id,products,reviews,app_id,helpful,funny,date,is_recommended,hours,review_id
32,4834290,433,36,493340,3,0,2019-04-20,0,10.7,857781
36,4834290,433,36,306130,2,0,2017-01-14,0,37.3,5372733
38,4834290,433,36,1509510,0,0,2021-12-12,0,1.3,6133318
39,4834290,433,36,1258080,4,0,2020-05-05,0,2.1,6258447
40,4834290,433,36,1385380,0,0,2021-04-23,0,19.1,6404588
...,...,...,...,...,...,...,...,...,...,...
429414,922219,72,25,1172620,0,0,2021-12-28,0,4.6,3264332
429416,922219,72,25,1172470,0,0,2021-12-28,0,6.3,3538330
429418,922219,72,25,1449850,3,0,2022-06-26,0,1.7,4088965
429420,922219,72,25,252490,0,0,2021-12-30,0,4.3,4791579


In [132]:
reader = sp.Reader(rating_scale=(0, 1))
data = sp.Dataset.load_from_df(user_app_ratings[["user_id", "app_id", "is_recommended"]], reader)
data

In [133]:
from surprise.model_selection import train_test_split

#uid = 1965432
#iid = 264710

trainset, testset = train_test_split(data, test_size=0.25, random_state=1)

In [41]:
from surprise import KNNBasic

# User-based with Pearson correlation similarity and simple prediction
sim_options = {'name': 'pearson', 'user_based': True, 'min_support': 1}

# Build algorithm
algo = KNNBasic(sim_options=sim_options, k=10)
algo.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [42]:
uid = 4834290
iid = 493340

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=0, verbose=True)

user: 4834290    item: 493340     r_ui = 0.00   est = 0.00   {'actual_k': 10, 'was_impossible': False}


In [43]:
from surprise.accuracy import mse, rmse, mae
# Compute MSE, RMSE and MAE on the test set predictions

predictions = algo.test(testset)

mse(predictions)
rmse(predictions)
mae(predictions)

MSE: 0.1752
RMSE: 0.4186
MAE:  0.3226


0.3225770509367906

# SVD

In [44]:
from surprise import SVD

algo = SVD(n_epochs=10, lr_all=0.005, reg_all=0.4)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute MSE, RMSE and MAE
mse(predictions)
rmse(predictions)
mae(predictions)

MSE: 0.1307
RMSE: 0.3615
MAE:  0.2790


0.27899567356367966

# SVD++

In [134]:
from surprise import SVDpp

algo_svd_pp = SVDpp()

algo_svd_pp.fit(trainset)
predictions_svdpp = algo_svd_pp.test(testset)

# Then compute MSE, RMSE and MAE
mse(predictions_svdpp)
rmse(predictions_svdpp)
mae(predictions_svdpp)

MSE: 0.1263
RMSE: 0.3554
MAE:  0.2565


0.2564551599544896

In [168]:
algo_svd_pp.predict(1965432, 1313140, r_ui=0, verbose=True)

user: 1965432    item: 1313140    r_ui = 0.00   est = 1.00   {'was_impossible': False}


Prediction(uid=1965432, iid=1313140, r_ui=0, est=1, details={'was_impossible': False})

In [147]:
for uid, iid, true_r, est, _ in predictions:
    if uid == 4895596:
        print(uid, iid, true_r, est)

4895596 377160 1.0 0.7469714386054233
4895596 232090 1.0 0.8352889955005722
4895596 215530 0.0 0.8039444253749296
4895596 858820 1.0 0.5734995030464294
4895596 965200 1.0 0.7820919216198077
4895596 548430 1.0 0.9828030774341352


In [149]:
trainset2 = data.build_full_trainset()

In [178]:
### NEW ###
trainset_new, testset_new = train_test_split(data, test_size=0.25, random_state=1)

In [192]:
## new
algo_svd_pp3 = SVDpp()
algo_svd_pp3.fit(trainset_new)
testset_new1 = trainset_new.build_anti_testset()
predictions_svdpp3 = algo_svd_pp3.test(testset_new1)

In [189]:
trainset_new.n_items

1782

In [190]:
len(set([i for (u,i,r) in testset_new]))

1697

In [150]:
algo_svd_pp2 = SVDpp()
algo_svd_pp2.fit(trainset2)

In [151]:
testset2 = trainset2.build_anti_testset()

In [152]:
predictions_svdpp2 = algo_svd_pp2.test(testset2)

In [153]:
for uid, iid, true_r, est, _ in predictions_svdpp2:
    if uid == 4895596:
        print(uid, iid, true_r, est)

4895596 264710 0.7887470222362977 0.9180614970350015
4895596 239030 0.7887470222362977 0.9599624868106139
4895596 250900 0.7887470222362977 0.9464687991599671
4895596 607080 0.7887470222362977 1
4895596 335300 0.7887470222362977 0.7884182790434784
4895596 860510 0.7887470222362977 0.9079729833006619
4895596 200510 0.7887470222362977 0.9396994857506165
4895596 527230 0.7887470222362977 0.9011491979131815
4895596 379430 0.7887470222362977 0.9181860584745104
4895596 504230 0.7887470222362977 0.9474296214716653
4895596 1366540 0.7887470222362977 1
4895596 1092790 0.7887470222362977 0.9707544812329423
4895596 220200 0.7887470222362977 0.8956953166180528
4895596 203770 0.7887470222362977 0.8021656687060243
4895596 617830 0.7887470222362977 0.777747538976933
4895596 927380 0.7887470222362977 0.9896272027325793
4895596 1049410 0.7887470222362977 0.9554359339735106
4895596 916840 0.7887470222362977 0.7828665204431107
4895596 362890 0.7887470222362977 0.9882974433508247
4895596 501300 0.78874702

In [135]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [136]:
top_n = get_top_n(predictions_svdpp, n=10)

# Let's get recommendations for user 196
#uid = 4895596

# collect all movies the user rated
#user_rated_apps = user_app_ratings.loc[user_app_ratings['user_id'] == uid]['app_id']

# get the recommendations for the specific user from the top_n dict
#recommendations = [user_app_ratings[user_app_ratings['app_id'] == iid)]['title'].iloc[0] for (iid, _) in top_n[uid]]
#recommendations = [x for x in ]

#print('Top 5 recommendations for user with id 196:')
#for i, rec in enumerate(recommendations):
#    print(f'{i+1} {rec}')

In [167]:
get_top_n(predictions_svdpp2, n=50)

defaultdict(list,
            {1965432: [(1313140, 1),
              (526870, 1),
              (1385380, 1),
              (1169040, 1),
              (646570, 1),
              (1432050, 1),
              (457140, 1),
              (758370, 1),
              (233860, 1),
              (1007040, 1),
              (375480, 1),
              (410320, 1),
              (1127400, 1),
              (595520, 1),
              (329130, 1),
              (298030, 1),
              (1332010, 1),
              (546560, 1),
              (431960, 1),
              (1336490, 1),
              (972660, 1),
              (1057090, 1),
              (1125240, 1),
              (615120, 1),
              (1408230, 1),
              (1598160, 1),
              (1794680, 1),
              (620980, 1),
              (548430, 1),
              (975370, 1),
              (294100, 1),
              (22330, 1),
              (49520, 1),
              (1235140, 1),
              (528230, 1),
              (8

In [170]:
[(a,b,c) for (a,b,c,d) in data.raw_ratings if a == 1965432]

[(1965432, 264710, 1.0),
 (1965432, 239030, 1.0),
 (1965432, 250900, 1.0),
 (1965432, 607080, 1.0),
 (1965432, 335300, 1.0),
 (1965432, 860510, 1.0),
 (1965432, 200510, 1.0),
 (1965432, 527230, 1.0),
 (1965432, 379430, 1.0),
 (1965432, 504230, 1.0),
 (1965432, 1366540, 1.0),
 (1965432, 1092790, 1.0),
 (1965432, 220200, 1.0),
 (1965432, 203770, 1.0),
 (1965432, 617830, 1.0),
 (1965432, 927380, 1.0),
 (1965432, 1049410, 1.0),
 (1965432, 916840, 1.0),
 (1965432, 362890, 1.0),
 (1965432, 501300, 1.0),
 (1965432, 424840, 1.0),
 (1965432, 597220, 1.0),
 (1965432, 788690, 1.0),
 (1965432, 334940, 1.0),
 (1965432, 15390, 1.0),
 (1965432, 890720, 1.0),
 (1965432, 530320, 1.0),
 (1965432, 251130, 1.0),
 (1965432, 206190, 1.0),
 (1965432, 588950, 1.0),
 (1965432, 330020, 1.0),
 (1965432, 780290, 1.0)]

In [197]:
def get_recommendation_for_user(uid, top_n, testset):
    recommended = pd.Series([(x,y) for (x,y) in top_n[uid]])
    #print(recommended)
    in_testset = pd.Series([i for (u, i, r) in testset if u == uid])

    print ("\nUser: " + str(uid))
    print("Relevant Rated (Testset):")
    counter = 1
    for i in in_testset:
        print(str(counter) + "- "+ str(i) + '- ' + games[games['app_id'] == i]['title'].iloc[0])
        counter+=1

    print("\n Recommended Items:")
    counter = 1
    for r in recommended:
        #print(str(counter) + '- ' + i)
        print(str(counter) + "- " + str(r) + '- ' +  games[games['app_id'] == r[0]]['title'].iloc[0])
        counter+=1


get_recommendation_for_user(4895596, get_top_n(predictions_svdpp3, n=2000), testset_new)


User: 4895596
Relevant Rated (Testset):
1- 237990- The Banner Saga
2- 292030- The Witcher® 3: Wild Hunt
3- 858820- Tribes of Midgard
4- 55150- Warhammer 40000: Space Marine - Anniversary Edition

 Recommended Items:
1- (698540, 1)- Dungeon Warfare 2
2- (876650, 1)- Aground
3- (1150690, 1)- OMORI
4- (851670, 1)- LUMINES REMASTERED
5- (615120, 1)- COMPOUND
6- (304410, 1)- Hexcells Infinite
7- (792100, 1)- 7 Billion Humans
8- (1255650, 1)- Geometry Arena
9- (477740, 1)- Zero Escape: The Nonary Games
10- (1046400, 1)- Astalon: Tears of the Earth
11- (511470, 1)- Glass Masquerade
12- (702050, 1)- The Song of Saya
13- (603960, 1)- Monolith
14- (367670, 1)- Controller Companion
15- (1169040, 1)- Necesse
16- (1408230, 1)- Walkabout Mini Golf VR
17- (635940, 1)- Little Busters! English Edition
18- (1548750, 1)- Conway: Disappearance at Dahlia View
19- (284770, 1)- Enigmatis 2: The Mists of Ravenwood
20- (336140, 1)- Unavowed
21- (1639790, 1)- Northern Journey
22- (703320, 1)- Hidden & Dangerou

In [164]:
user_app_ratings.loc[user_app_ratings['user_id'] == 4895596]

,user_id,products,reviews,app_id,helpful,funny,date,is_recommended,hours,review_id
85079,4895596,389,37,1217060,35,0,2021-09-15,1,347.9,73474
85080,4895596,389,37,1290000,4,0,2021-09-01,1,0.6,240789
85081,4895596,389,37,1172620,25,5,2021-08-29,0,21.2,450840
85082,4895596,389,37,292030,5,0,2015-05-27,1,164.2,461963
85083,4895596,389,37,493520,4,0,2020-02-29,1,97.0,862365
85084,4895596,389,37,548430,9,0,2018-02-28,1,299.0,1138296
85085,4895596,389,37,1145360,20,0,2021-10-28,1,21.8,1183000
85086,4895596,389,37,495420,6,0,2020-03-14,1,91.8,1267752
85087,4895596,389,37,858820,29,4,2021-07-28,1,46.3,1727263
85088,4895596,389,37,1174180,3,0,2022-01-13,1,263.2,2150960


In [144]:
testset

[(5458269, 648800, 1.0),
 (5653074, 508440, 1.0),
 (3467583, 705220, 1.0),
 (5495255, 731490, 1.0),
 (3876654, 504230, 1.0),
 (5268160, 220, 1.0),
 (3120351, 1107790, 1.0),
 (6138064, 238960, 1.0),
 (1967973, 292030, 1.0),
 (5096603, 1113560, 1.0),
 (1122779, 678950, 1.0),
 (4801586, 305620, 1.0),
 (6128600, 1341290, 1.0),
 (5786199, 227600, 1.0),
 (2613306, 1508400, 0.0),
 (2374657, 839310, 0.0),
 (5210908, 564230, 1.0),
 (5221264, 275390, 1.0),
 (4012161, 262060, 1.0),
 (3859887, 860510, 1.0),
 (3031599, 1238810, 1.0),
 (886728, 204100, 1.0),
 (534732, 780310, 1.0),
 (922219, 1449850, 0.0),
 (4929497, 22320, 1.0),
 (140915, 379430, 1.0),
 (2830266, 1250410, 1.0),
 (588148, 271590, 1.0),
 (5171323, 262060, 1.0),
 (5481692, 678960, 0.0),
 (2907624, 1745510, 1.0),
 (1846686, 823500, 1.0),
 (120907, 220, 1.0),
 (5702222, 867210, 1.0),
 (2418587, 1147560, 1.0),
 (1997835, 438740, 1.0),
 (4208500, 582660, 1.0),
 (1573234, 703860, 1.0),
 (3077193, 12210, 1.0),
 (4739020, 397540, 0.0),
 (248

In [110]:
rrr = pd.Series([i for (u, i, r) in testset if u == uid])

for (u, i, r) in testset:
    if u == uid:
        print("User: " + str(u) + " App: " + str(i))

User: 4895596 App: 380600
User: 4895596 App: 979690
User: 4895596 App: 301640
User: 4895596 App: 275850
User: 4895596 App: 275390
User: 4895596 App: 305620
User: 4895596 App: 495420
User: 4895596 App: 397540
User: 4895596 App: 1145360
User: 4895596 App: 293260
User: 4895596 App: 292030
User: 4895596 App: 107410


In [106]:
rrr

0    976730
1    107410
dtype: int64

<bound method Trainset.all_ratings of <surprise.trainset.Trainset object at 0x28c90f580>>

In [111]:
r2 = rec[rec.isin(rrr)]
r2

2    292030
dtype: int64

In [ ]:
from collections import defaultdict

from surprise import Dataset, SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k=10, threshold=0.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        #if precisions[uid] < 1 and precisions[uid] > 0.5:
        #    print(uid)

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


#data = Dataset.load_builtin("ml-100k")


In [208]:
precisions, recalls = precision_recall_at_k(predictions_svdpp3, k=10, threshold=0.78)
print('MAP@10: '+ str(sum(prec for prec in precisions.values()) / len(precisions)))

MAP@10: 0.9316662209146831


In [209]:
kf = KFold(n_splits=5)
algo = SVDpp()

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=0.5)

    # Precision and recall can then be averaged over all users
    print('MAP@10: '+ str(sum(prec for prec in precisions.values()) / len(precisions)))
    print(sum(rec for rec in recalls.values()) / len(recalls))

5908817
4873103
5257556
2136804
5794995
1344015
5207229
1338999
5770687
4422053
2698223
3894928
5226320
2088877
1271586
803811
5017825
186295
339356
5956542
3968133
4341856
1627833
4495163
4019045
2772337
5350691
2756154
461719
5006513
3474502
4153414
5046300
1274583
4962866
4872723
4921667
1528116
1887731
4421160
3000539
3352145
566953
4208040
5724175
3343391
3843605
4391171
2070406
4709423
2179502
5511655
3672414
3961893
4867108
6028260
4986648
751087
254709
1993957
1918700
4694629
4981563
2084011
3777137
2476868
4684682
5436996
3698086
5568510
4413963
5709100
3653608
1821557
5811964
2686413
1313040
1395607
2254777
2494364
4975243
4237977
1672486
2325374
5112060
1995392
1384837
2762030
5309334
2595065
1817035
5126841
3771743
3579483
448406
3237348
3127777
2279300
6082418
3270506
3896524
5036566
4700104
5072664
2134554
4693985
3505675
440920
4552765
4293096
3427466
4050683
2198119
2137848
2585806
5744506
1033710
254575
4549865
4046072
546550
730363
3229525
3805921
4821668
3753752
3687

KeyboardInterrupt: 